## Pydantic v2: continue

### Field Validator
Field validator is for specific field in class attribute

In [8]:
from pydantic import BaseModel, EmailStr, field_validator

class Owner(BaseModel):
    name: str
    email: EmailStr
    
    @field_validator("name")
    @classmethod
    def name_must_contain_space(cls, v):
        if " " not in v:
            raise ValueError("must contain a space in name field")
        return v.upper() # name will be converted to uppercase letter
    
print("For owner1")
try:
    owner = Owner(
        name ="sam",
        email = "sam@gmail.com"
    )
except ValueError as e:
    print(e)
    
print("for owner2")
try:
    owner = Owner(
        name = "sam smith",
        email = "samsmith@xyz.com"
    )
    print(owner)
except ValueError as e:
    print(e)

For owner1
1 validation error for Owner
name
  Value error, must contain a space in name field [type=value_error, input_value='sam', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/value_error
for owner2
name='SAM SMITH' email='samsmith@xyz.com'


### Model Validator
validate a complete data model

In [16]:
from typing import Any
from pydantic import EmailStr, ValidationError, model_validator, BaseModel

class Owner(BaseModel):
    name: str
    email: EmailStr
    
    @model_validator(mode='before') # run before checking for data type and all
    @classmethod
    def check_sensitive_info(cls, data: Any):
        print(f"Received data: {data}")
        if isinstance(data, dict):
            if "password" in data:
                raise ValueError("password shold not be included.")
            if "card_number" in data:
                raise ValueError("card_number should not be included.")
        return data
    
    @model_validator(mode="after") # after the data type is checked
    def check_name_contain_space(self):
        if " " not in self.name:
            raise ValueError("must contain space in name")
        return self
    
try:
    owner = Owner(
        name="sam",
        email="name@xyz.com", # even if there is validation error
        password="1234" # this run because of "before"
    )
    print(owner)
except ValidationError as e:
    print(e)
    
print("\n")

try:
    owner = Owner(
        name="alice", # there is no error for sensitive check so validator after will run
        email="email@xyz.com"
    )
    print(owner)
except Exception as e:
    print(e) 
    
print("\n")

try:
    owner = Owner(
        name = "John Doe",
        email = "john@xyz.com"
    )
    print(f"Owner detail: {owner} and type: {type(owner)}")
except ValidationError as e:
    print(e)

Received data: {'name': 'sam', 'email': 'name@xyz.com', 'password': '1234'}
1 validation error for Owner
  Value error, password shold not be included. [type=value_error, input_value={'name': 'sam', 'email': ...om', 'password': '1234'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.7/v/value_error


Received data: {'name': 'alice', 'email': 'email@xyz.com'}
1 validation error for Owner
  Value error, must contain space in name [type=value_error, input_value={'name': 'alice', 'email': 'email@xyz.com'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.7/v/value_error


Received data: {'name': 'John Doe', 'email': 'john@xyz.com'}
Owner detail: name='John Doe' email='john@xyz.com' and type: <class '__main__.Owner'>


### Field

In [18]:
from pydantic import Field, BaseModel
from uuid import uuid4

class User(BaseModel):
    id: int = Field(default_factory=lambda: uuid4().hex) # default_factory is used to generate value at runtime

user = User()
print(user.model_dump())

{'id': '70575b5417e747049dc6ca15afebb332'}


C:\Users\acer\AppData\Roaming\Python\Python312\site-packages\pydantic\main.py:347: UserWarning: Pydantic serializer warnings:
  Expected `int` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


In [19]:
from pydantic import Field

class User(BaseModel):
    name: str = Field(..., alias="username")
    
user = User(username="john")
print(user)
print(user.model_dump(by_alias=True))

name='john'
{'username': 'john'}


### Field Constraint

In [23]:
from typing import List
from pydantic import BaseModel, Field, EmailStr
from decimal import Decimal

class User(BaseModel):
    username: str = Field(..., min_length=3, max_length=50, pattern=r"^\w+$") # this pattern is for word character only
    email: EmailStr = Field(...)
    age: int = Field(..., gt=0, lt=110)
    balance: Decimal = Field(..., max_digits=10, decimal_places=2)
    is_active: bool = Field(True)
    favourite_numbers: List[int] = Field(..., min_items=1)
    height: float = Field(..., gt=0.0)
    
user = User(
    username="john",
    email="example@gmail.com",
    age=10,
    balance=100.0,
    favourite_numbers=[1,2,3],
    height=6.8,
    is_active=True
)

user

User(username='john', email='example@gmail.com', age=10, balance=Decimal('100.0'), is_active=True, favourite_numbers=[1, 2, 3], height=6.8)

In [25]:
from pydantic import BaseModel, computed_field
from datetime import datetime

class Person(BaseModel):
    name: str
    birth_year: int
    
    @computed_field # compute a new field based on the existing field
    @property
    # property is used to make a method as attribute
    def age(self) -> int:
        current_year = datetime.now().year
        return current_year - self.birth_year
    
person = Person(
    name="John",
    birth_year=1990
)

person

Person(name='John', birth_year=1990, age=34)

In [40]:
from pydantic import BaseModel, field_validator
from datetime import datetime

class Person(BaseModel):
    name: str
    birth_year: int
    
    @computed_field
    @property
    def age(self) -> int:
        current_year = datetime.now().year
        age = current_year - self.birth_year
        return age
    
    @field_validator("birth_year")
    @classmethod
    def validate_age(cls, v:int):
        current_year = datetime.now().year
        if current_year - v < 18:
            raise ValueError("age must be greater than 18")
        return v
    
try:
    Person(name="John", birth_year=2020)
except ValidationError as e:
    print(e)

person = Person(name="John", birth_year=1960)
person

1 validation error for Person
birth_year
  Value error, age must be greater than 18 [type=value_error, input_value=2020, input_type=int]
    For further information visit https://errors.pydantic.dev/2.7/v/value_error


Person(name='John', birth_year=1960, age=64)